In [ ]:
import pandas as pd
import utils.databases as db
import pytz
#import calendar
melbourne_tz = pytz.timezone('Australia/Melbourne')

#connection = db.get_db_connection()
sensor_locations = '''select * from open_data_ingestion_pedestrian_counting_system_sensor_locations'''
sensor_locations_df = db.get_query_df(sensor_locations)

counts = '''select * from open_data_ingestion_pedestrian_counting_system_monthly_counts_per_hour'''
counts_df = db.get_query_df(counts)
counts_df = counts_df[['timestamp', 'total_of_directions','locationid']]
counts_df['timestamp'] = pd.to_datetime(counts_df['timestamp'], format='%Y-%m-%dT%H:%M:%S%z', errors='coerce')
counts_df['timestamp'] = counts_df['timestamp'].dt.tz_convert(melbourne_tz)
counts_df['timestamp'] = counts_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')


archive_count = pd.read_csv('historical_data/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv')
archive_count['Date_Time'] = pd.to_datetime(archive_count['Date_Time'], format='%B %d, %Y %I:%M:%S %p', errors='coerce')
new_column_names = {'Sensor_ID': 'locationid', 'Date_Time': 'timestamp', 'Hourly_Counts': 'total_of_directions'}
archive_count = archive_count.rename(columns=new_column_names)
counts_df = pd.concat([archive_count, counts_df], ignore_index= True)

counts_df['timestamp'] = pd.to_datetime(counts_df['timestamp'])
counts_df['Day'] = counts_df['timestamp'].dt.day_name()
#counts_df['Day'] = pd.to_datetime(counts_df['Day'], format='%w').dt.strftime('%A')
counts_df['Mdate'] = counts_df['timestamp'].dt.day
counts_df['Month'] = counts_df['timestamp'].dt.month
counts_df['Year'] = counts_df['timestamp'].dt.year
counts_df['Time'] = counts_df['timestamp'].dt.hour
#counts_df = counts_df[['locationid','timestamp','Year', 'Month', 'Time', 'Sensor_Name', 'total_of_directions']]

joined_results = pd.merge(counts_df, sensor_locations_df, left_on='locationid', right_on='location_id', how='left')
joined_results['Sensor_Name'].fillna(joined_results['sensor_description'], inplace=True)
joined_results = joined_results[['locationid','timestamp','Year', 'Day', 'Mdate', 'Month', 'Time', 'Sensor_Name', 'total_of_directions']]


: 

In [10]:
# Top 10 locations by day
top_10_locations_day = joined_results.groupby(['locationid', 'Day'])['total_of_directions'].sum().reset_index()
top_10_locations_day = top_10_locations_day.groupby('Day').apply(lambda x: x.nlargest(10, 'total_of_directions')).reset_index(drop=True)
top_10_locations_day = top_10_locations_day.sort_values(by=['Day', 'total_of_directions'], ascending=[True, False])
top_10_locations_day

,locationid,Day,total_of_directions
0,4,Friday,24863860
1,6,Friday,21787762
2,3,Friday,20604592
3,2,Friday,19217529
4,5,Friday,18975538
...,...,...,...
65,1,Wednesday,15573172
66,24,Wednesday,13531002
67,22,Wednesday,12031778
68,15,Wednesday,11000867


In [14]:
top_locations_each_day = top_10_locations_day.groupby('Day').max('total_of_directions').reset_index()
top_locations_each_day

,Day,locationid,total_of_directions
0,Friday,35,24863860
1,Monday,24,19744581
2,Saturday,35,23142992
3,Sunday,35,18693998
4,Thursday,24,21597142
5,Tuesday,24,19884050
6,Wednesday,24,20808541


In [15]:
# Top 10 locations by month
joined_results['Month'] = joined_results['timestamp'].dt.month
top_10_locations_month = joined_results.groupby(['locationid', 'Month'])['total_of_directions'].sum().reset_index()
top_10_locations_month = top_10_locations_month.groupby('Month').apply(lambda x: x.nlargest(10, 'total_of_directions')).reset_index(drop=True)
top_10_locations_month

,locationid,Month,total_of_directions
0,4,1,11967639
1,5,1,10796073
2,6,1,10279628
3,2,1,9106886
4,3,1,8433264
...,...,...,...
115,3,12,9958892
116,22,12,7643186
117,35,12,7087552
118,15,12,6079839


In [16]:
# Location with most decline due to lockdowns in the last 3 years
lockdown_start_date = pd.to_datetime('2020-03-01')
lockdown_end_date = pd.to_datetime('2023-03-01')
lockdown_data = joined_results[(joined_results['timestamp'] >= lockdown_start_date) & (joined_results['timestamp'] < lockdown_end_date)]
decline_location = lockdown_data.groupby('locationid')['total_of_directions'].sum().idxmin()
joined_results.Sensor_Name[joined_results.locationid == decline_location].iloc[0]

In [26]:
# Location with most growth in the last year
growth_start_date = pd.to_datetime('2022-06-29')
growth_end_date = pd.to_datetime('2023-06-29')
last_year = joined_results[joined_results['timestamp'].dt.year >= 2022]
growth_location = last_year.groupby('locationid').sum('total_of_directions')['total_of_directions'].idxmax()
joined_results.Sensor_Name[joined_results.locationid == growth_location].iloc[0]